# Hartree Fock Program V1

In [1]:
import psi4
import math
import numpy as np
from scipy import special
from scipy import linalg

import matplotlib.pyplot as plt

# Set Psi4 options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)
#psi4.set_num_threads(4);


  Memory set to   1.863 GiB by Python driver.


In [2]:
class BasisInfo:
    def __init__(self, basis_set):
        self.basis_set = basis_set
        self.nbf = basis_set.nbf()
    
    def print_info(self):
        
        nbf=self.basis_set.nbf();
        print("Number of basis functions: ", nbf, "\n")

        for i in range(self.nbf):
            shell = self.basis_set.shell(i)
            num_prim = shell.nprimitive

            for j in range(num_prim):
                coefficient = shell.coef(j)
                exponent = shell.exp(j)

                print(f"Basis Function {i+1}, Component {j+1}: ")
                print("Coefficient: ", coefficient)
                print("Exponent: ", exponent, "\n")

In [3]:
class DIIS:
    def __init__(self, max_vecs=6):
        self.error_vecs = []
        self.fock_matrices = []
        self.max_vecs = max_vecs

    def add(self, F, D, S):
        """오차 벡터와 Fock 행렬 추가"""
        error = np.dot(F, D).dot(S) - np.dot(S, D).dot(F)
        self.error_vecs.append(error)
        self.fock_matrices.append(F.copy())

        # 최대 벡터 수 초과 시 가장 오래된 것 제거
        if len(self.error_vecs) > self.max_vecs:
            del self.error_vecs[0]
            del self.fock_matrices[0]

    def extrapolate(self):
        """DIIS Fock 행렬 계산 및 반환"""
        n = len(self.error_vecs)
        if n < 2:
            return self.fock_matrices[-1]

        # B 행렬 구성
        B = np.empty((n + 1, n + 1))
        B[-1, :] = -1
        B[:, -1] = -1
        B[-1, -1] = 0
        for i in range(n):
            for j in range(n):
                B[i, j] = np.vdot(self.error_vecs[i], self.error_vecs[j])

        # 선형 방정식 풀기
        rhs = np.zeros(n + 1)
        rhs[-1] = -1
        coeffs = np.linalg.solve(B, rhs)

        # Fock 행렬 조합
        F_diis = np.zeros_like(self.fock_matrices[0])
        for i in range(n):
            F_diis += coeffs[i] * self.fock_matrices[i]

        return F_diis

In [5]:
with open('./results/butane_cc-pVDZ_dihedral_test.txt', 'w') as file:
    #distances = np.linspace(1, 2, 50);  # example range and number of points
    distances = np.linspace(1.54, 1.54, 1);  # example range and number of points
    
    #angles = np.linspace(60, 300, 49)  # example range and number of points
    #angles = np.linspace(90, 140, 101)
    angles = np.linspace(-10, 370, 77)
    #angles = np.linspace(60, 60, 1)  
    
    max_iter = 1000
    tolerance = 1e-5
    
    for dist in distances: 
    
        for ang in angles:
            
            print(dist, ang)

            
            # 1) specify a molecule 
            molecule = psi4.geometry(f"""
            0 1
            C  
            C 1 1.54
            C 2 1.54 1 112.7
            C 3 1.54 2 112.7 1 {ang}
            H 1 1.09 2 109.5 3 60
            H 1 1.09 2 109.5 3 300
            H 1 1.09 2 109.5 3 180
            H 2 1.09 1 109.5 3 -121.4
            H 2 1.09 1 109.5 3 121.4
            H 3 1.09 2 109.5 1 {ang-121.4}
            H 3 1.09 2 109.5 1 {ang+121.4}
            H 4 1.09 3 109.5 2 300
            H 4 1.09 3 109.5 2 180
            H 4 1.09 3 109.5 2 60
            """)
            
            
            # Set up the basis set
            #basis = '6-31G'
            #basis = '6-31G(d,p)'
            #basis = '6-311G(d,p)'
            #basis = '6-311+G(d,p)'
            #basis = '6-311G(2d,2p)'
            #basis = '6-31G*'
            basis = 'cc-pVDZ'
            #basis = 'cc-pVQZ'
            basis_set = psi4.core.BasisSet.build(molecule, 'BASIS', basis);

            nbf=basis_set.nbf();
            print("Number of basis functions: ", nbf, "\n");
            
            pdb_string = molecule.save_string_xyz_file();    
            
            with open(f"./mol_image_propane/molecule_butane_{dist:.4}_{ang:.4}.xyz", 'w') as f:
                f.write(pdb_string)
                
            # Calculate integrals     

            # Create MintsHelper
            mints = psi4.core.MintsHelper(basis_set);

            # Compute overlap integral   
            S = np.asarray(mints.ao_overlap());

            # Compute Nuclear attraction integral 
            T = np.asarray(mints.ao_kinetic());

            # Compute potential energy integral  
            V = np.asarray(mints.ao_potential());

            # Form core Hamiltonian integral 
            H = T+V;

            # Compute two-electron repulsion integrals 
            eri = np.asarray(mints.ao_eri());

            #Nuclear repulsion energy 
            E_nucl = molecule.nuclear_repulsion_energy();

            # Number of occupied orbital
            num_protons = sum([molecule.Z(i) for i in range(molecule.natom())]);
            num_electrons = num_protons - abs(molecule.molecular_charge());
            num_occupied_orbitals =int(num_electrons//2);
            
            # Diagonalize the S matrix and construct X
            eigvals, eigvecs = linalg.eigh(S)
            eigvals_diag = np.diag(1 / np.sqrt(eigvals))
            #X = eigvecs @ eigvals_diag @ eigvecs.T
            X = eigvecs @ eigvals_diag
            
            # Initial guess of density matrix 
            E_core, C_core = np.linalg.eigh(H)
            C=C_core[:,:num_occupied_orbitals];
            P = 2.0 * C @ C.T
            
            
            diis_helper = DIIS()

            E_elec=0; pre_E_elec=0; P_pre = np.zeros((nbf, nbf))

            for iteration in range(max_iter):

                ## G calculation 
                J = np.einsum("pqrs,rs->pq", eri, P)
                K = np.einsum("prqs,rs->pq", eri, P)
                G=J-0.5*K;
                F=H+G;
                
                diis_helper.add(F, P, S)
                
                if iteration>1:
                    F=diis_helper.extrapolate()
                
                F_prime = X.T@F@X
                # Compute eigenvalues and eigenvectors
                eigenvalues, C_prime = linalg.eigh(F_prime);
                
                C = X@C_prime
                P = 2 * C[:,:num_occupied_orbitals] @ C[:,:num_occupied_orbitals].T

               
                # Calculate energy 
                E_elec = np.sum(H*P) + 0.5*np.sum(G*P)
                E_total = E_elec + E_nucl
                #print("SCF converged in %d iterations." % (iteration+1))
                #print("dist: ", dist, ", angle: ", ang, ", E: ", E_total);
                
                delta_P = np.sqrt(np.sum((P-P_pre)**2))
                delta_E = abs(E_elec - pre_E_elec)
                
                # Update old density matrix for the next iteration 
                P_pre = P.copy()

                if delta_E < tolerance and delta_P < tolerance:
                    break
                    
                pre_E_elec=E_elec

    #             print("SCF converged in %d iterations." % (iteration+1))
    #             print("Electronic energy: %.8f a.u." % E_elec)
    #             print("Nuclear repulsion energy: %.8f a.u." % E_nucl)
    #             print("Total energy: %.8f a.u." % E_total)
    
            energy = psi4.energy('scf/cc-pVDZ', molecule=molecule)
            print("SCF converged in %d" % (iteration+1), ", ang: ", ang, ", E: ", E_total, ", ref E: ", energy, "\n");
            file.write(f"{ang:.4f}, {E_total:.8f}, {energy:.6f}\n")

        # #energies.append(E_total)


print("Done !\n");

#basis_info=BasisInfo(basis_set)
#basis_info.print_info()



1.54 -10.0
Number of basis functions:  106 

SCF converged in 14 , ang:  -10.0 , E:  -157.29506067829377 , ref E:  -157.295018571604 

1.54 -5.0
Number of basis functions:  106 

SCF converged in 14 , ang:  -5.0 , E:  -157.29432880445964 , ref E:  -157.29428803715194 

1.54 0.0
Number of basis functions:  106 

SCF converged in 14 , ang:  0.0 , E:  -157.29407168806821 , ref E:  -157.29403144527902 

1.54 5.0
Number of basis functions:  106 

SCF converged in 14 , ang:  5.0 , E:  -157.29432880445404 , ref E:  -157.29428803719955 

1.54 10.0
Number of basis functions:  106 

SCF converged in 14 , ang:  10.0 , E:  -157.295060678293 , ref E:  -157.29501857159897 

1.54 15.0
Number of basis functions:  106 

SCF converged in 14 , ang:  15.0 , E:  -157.29616214470934 , ref E:  -157.29611832717646 

1.54 20.0
Number of basis functions:  106 

SCF converged in 14 , ang:  20.0 , E:  -157.29749507233393 , ref E:  -157.29744951916877 

1.54 25.0
Number of basis functions:  106 

SCF converged in 

SCF converged in 16 , ang:  295.0 , E:  -157.3065856397721 , ref E:  -157.30653823130322 

1.54 300.0
Number of basis functions:  106 

SCF converged in 15 , ang:  300.0 , E:  -157.30622716856536 , ref E:  -157.3061819485057 

1.54 305.0
Number of basis functions:  106 

SCF converged in 15 , ang:  305.0 , E:  -157.30566024075185 , ref E:  -157.30561449120572 

1.54 310.0
Number of basis functions:  106 

SCF converged in 15 , ang:  310.0 , E:  -157.30490316197972 , ref E:  -157.30485712280637 

1.54 315.0
Number of basis functions:  106 

SCF converged in 15 , ang:  315.0 , E:  -157.3039770920019 , ref E:  -157.30393066174497 

1.54 320.0
Number of basis functions:  106 

SCF converged in 14 , ang:  320.0 , E:  -157.30290081472825 , ref E:  -157.30285129837938 

1.54 325.0
Number of basis functions:  106 

SCF converged in 14 , ang:  325.0 , E:  -157.30168189433545 , ref E:  -157.301632802011 

1.54 330.0
Number of basis functions:  106 

SCF converged in 14 , ang:  330.0 , E:  -157.3